In [127]:
_input_path = analysis_dir / "combined_evidence_filter.json"
with _input_path as _f:
    combined_evidence_filter = json.load(_f)

In [ ]:
filter_type_ordered = py_.chain(THRESHOLDS).map(lambda e: e["label"]).value()
evidence_group_ordered = ["assoc", "triple"]
evidence_type_ordered = [
    "Supporting",
    "Reversal",
    "Insufficient (directional)",
    "Insufficient (non-directional)",
    "Addtional (directional)",
]


----

# evidence items

In [201]:
# reformat evidence into individual score df
def make_individual_scores_df(evidence):
    evidence_df = pd.DataFrame(evidence)
    individual_scores_df = evidence_df[
        [
            "doi",
            "subject_term",
            "object_term",
            "pred_term",
            "assoc_evidence",
            "triple_evidence",
        ]
    ].assign(
        evidence_count=lambda df: df.apply(
            lambda row: py_.chain(
                {
                    "assoc": row["assoc_evidence"],
                    "triple": row["triple_evidence"],
                }
            )
            .map(
                lambda v, k: [
                    {
                        "doi": row["doi"],
                        "subject_term": row["subject_term"],
                        "object_term": row["object_term"],
                        "pred_term": row["pred_term"],
                        "evidence_type": EVIDENCE_LABELS[_k],
                        "strength_score": (
                            ([_["triple_score"] for _ in _v])
                            if k == "triple"
                            else ([_["assoc_score"] for _ in _v])
                        ),
                        "mapping_score": ([_["mapping_score"] for _ in _v]),
                        "evidence_score": ([_["evidence_score"] for _ in _v]),
                        "evidence_group": k,
                    }
                    for _k, _v in v.items()
                ]
            )
            .reduce(lambda a, b: a + b)
            .apply(pd.DataFrame)
            .value(),
            axis=1,
        )
    )
    individual_scores_df = (
        py_.chain(
            pd.concat(individual_scores_df["evidence_count"].tolist())
            .assign(
                pred_group=lambda df: df["pred_term"].apply(
                    lambda x: epigraphdb.PRED_DIRECTIONAL_MAPPING[x]
                )
            )
            .to_dict(orient="records")
        )
        .map(
            lambda items: [
                {
                    "doi": items["doi"],
                    "subject_term": items["subject_term"],
                    "object_term": items["object_term"],
                    "pred_term": items["pred_term"],
                    "evidence_type": items["evidence_type"],
                    "evidence_group": items["evidence_group"],
                    "evidence_score": items["evidence_score"][idx],
                    "strength_score": items["strength_score"][idx],
                    "mapping_score": items["mapping_score"][idx],
                }
                for idx, _ in enumerate(items["evidence_score"])
            ]
        )
        .flatten()
        .apply(pd.DataFrame)
        .value()
    )
    individual_scores_df.info()
    return individual_scores_df


_df = make_individual_scores_df(
    py_.chain(combined_evidence_filter)
    .filter(lambda coll: coll["filter_type"] == "epigraphdb")
    .map(lambda coll: coll["evidence"])
    .nth(0)
    .value()[:100]
)
_df.info()
_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   doi             41 non-null     object 
 1   subject_term    41 non-null     object 
 2   object_term     41 non-null     object 
 3   pred_term       41 non-null     object 
 4   evidence_type   41 non-null     object 
 5   evidence_group  41 non-null     object 
 6   evidence_score  41 non-null     float64
 7   strength_score  41 non-null     float64
 8   mapping_score   41 non-null     float64
dtypes: float64(3), object(6)
memory usage: 3.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   doi             41 non-null     object 
 1   subject_term    41 non-null     object 
 2   object_term     41 non-null     object 
 3   pred_term       41 

,doi,subject_term,object_term,pred_term,evidence_type,evidence_group,evidence_score,strength_score,mapping_score
0,10.1101/2020.01.08.20016980,Diabetes Mellitus,Obesity,COEXISTS_WITH,Supporting,triple,2.301030,2.301030,1.0
1,10.1101/2020.02.06.20020974,Coronavirus Infections,Acute respiratory disease,CAUSES,Supporting,triple,1.477121,1.477121,1.0
2,10.1101/2020.02.24.20027474,Coronavirus Infections,Acute infectious disease,CAUSES,Supporting,triple,1.000000,1.000000,1.0
3,10.1101/2020.03.04.20030395,Coronavirus Infections,Disease,CAUSES,Supporting,triple,2.278754,2.278754,1.0
4,10.1101/2020.03.04.20030395,Coronavirus Infections,Disease,CAUSES,Reversal,triple,1.000000,1.000000,1.0


In [202]:
individual_scores_df = (
    py_.chain(combined_evidence_filter)
    .map(
        lambda coll: make_individual_scores_df(coll["evidence"]).assign(
            filter_type=coll["filter_type"]
        )
    )
    .apply(lambda coll: pd.concat(coll).reset_index(drop=True))
    .value()
)
individual_scores_df.info()

_output_path = analysis_dir / "individual_scores.csv"

with _output_path.open("w") as _f:
    individual_scores_df.to_csv(_f, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   doi             150 non-null    object 
 1   subject_term    150 non-null    object 
 2   object_term     150 non-null    object 
 3   pred_term       150 non-null    object 
 4   evidence_type   150 non-null    object 
 5   evidence_group  150 non-null    object 
 6   evidence_score  150 non-null    float64
 7   strength_score  150 non-null    float64
 8   mapping_score   150 non-null    float64
dtypes: float64(3), object(6)
memory usage: 10.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   doi             156 non-null    object 
 1   subject_term    156 non-null    object 
 2   object_term     156 non-null    object 
 3   pred_term     

----

# summary on count of evidence items

In [241]:
_input_path = analysis_dir / "individual_scores.csv"
with _input_path.open() as _f:
    individual_scores_df = pd.read_csv(_input_path).assign(
        filter_type=lambda df: pd.Categorical(
            df["filter_type"],
            categories=filter_type_ordered,
            ordered=True,
        ),
        evidence_group=lambda df: pd.Categorical(
            df["evidence_group"],
            categories=evidence_group_ordered,
            ordered=True,
        ),
        evidence_type=lambda df: pd.Categorical(
            df["evidence_type"],
            categories=evidence_type_ordered,
            ordered=True,
        ),
    )
individual_scores_df

,doi,subject_term,object_term,pred_term,evidence_type,evidence_group,evidence_score,strength_score,mapping_score,filter_type
0,10.1101/2020.01.08.20016980,Diabetes Mellitus,Obesity,COEXISTS_WITH,Supporting,triple,2.301030,2.301030,1.000000,epigraphdb
1,10.1101/2020.02.06.20020974,Coronavirus Infections,Acute respiratory disease,CAUSES,Supporting,triple,1.477121,1.477121,1.000000,epigraphdb
2,10.1101/2020.02.24.20027474,Coronavirus Infections,Acute infectious disease,CAUSES,Supporting,triple,1.000000,1.000000,1.000000,epigraphdb
3,10.1101/2020.03.04.20030395,Coronavirus Infections,Disease,CAUSES,Supporting,triple,2.278754,2.278754,1.000000,epigraphdb
4,10.1101/2020.03.04.20030395,Coronavirus Infections,Disease,CAUSES,Reversal,triple,1.000000,1.000000,1.000000,epigraphdb
...,...,...,...,...,...,...,...,...,...,...
39757,10.1101/2021.12.22.21268161,Pancreatic carcinoma,"Diabetes Mellitus, Non-Insulin-Dependent",COEXISTS_WITH,Supporting,triple,0.646121,1.000000,0.646121,asq_default(0.7)
39758,10.1101/2021.12.22.21268161,Pancreatic carcinoma,"Diabetes Mellitus, Non-Insulin-Dependent",COEXISTS_WITH,Supporting,triple,0.821125,1.301030,0.631135,asq_default(0.7)
39759,10.1101/2021.12.22.21268161,Pancreatic carcinoma,"Diabetes Mellitus, Non-Insulin-Dependent",COEXISTS_WITH,Supporting,triple,0.705564,1.000000,0.705564,asq_default(0.7)
39760,10.1101/2021.12.22.21268161,Pancreatic carcinoma,"Diabetes Mellitus, Non-Insulin-Dependent",COEXISTS_WITH,Supporting,triple,0.915378,1.301030,0.703580,asq_default(0.7)


In [250]:
evidence_count_df = (
    individual_scores_df.groupby(["evidence_group", "filter_type", "evidence_type"])
    .size()
    .rename("value")
    .to_frame()
    .reset_index(level="evidence_type")
    .pivot(columns="evidence_type", values="value")
    .reset_index()
    .rename_axis(None, axis=1)
)
evidence_count_df

,evidence_group,filter_type,Supporting,Reversal,Insufficient (directional),Insufficient (non-directional),Addtional (directional)
0,assoc,epigraphdb,0,0,0,7,0
1,assoc,asq_0.99,1,0,0,7,0
2,assoc,asq_0.95,88,15,51,61,0
3,assoc,asq_0.90,160,35,184,434,0
4,assoc,asq_0.85,485,119,598,1565,0
5,assoc,asq_0.80,1000,208,1413,3314,0
6,assoc,asq_default(0.7),3887,595,3976,8800,0
7,triple,epigraphdb,109,34,0,0,0
8,triple,asq_0.99,113,35,0,0,0
9,triple,asq_0.95,260,78,0,0,0


----

# summary on weighted count (evidence scores) of evidence items

In [248]:
evidence_scores_df = individual_scores_df[
    ["filter_type", "evidence_type", "evidence_group", "evidence_score"]
]

In [249]:
evidence_scores_df.groupby(
    ["evidence_group", "filter_type", "evidence_type"]
).mean().reset_index(level="evidence_type").pivot(
    columns="evidence_type", values="evidence_score"
)

evidence_type                    Supporting  Reversal  \
evidence_group filter_type                              
assoc          epigraphdb               NaN       NaN   
               asq_0.99            1.437106       NaN   
               asq_0.95            1.949175  1.739306   
               asq_0.90            1.755730  1.686352   
               asq_0.85            1.446841  1.518254   
               asq_0.80            1.317440  1.409634   
               asq_default(0.7)    1.090855  1.176790   
triple         epigraphdb          1.862171  1.552502   
               asq_0.99            1.842481  1.536716   
               asq_0.95            1.622286  1.402430   
               asq_0.90            1.366801  1.202905   
               asq_0.85            1.243366  1.135382   
               asq_0.80            1.143683  1.040917   
               asq_default(0.7)    1.034755  0.930732   

evidence_type                    Insufficient (directional)  \
evidence_group filter_type                                    
assoc          epigraphdb                               NaN   
               asq_0.99                                 NaN   
               asq_0.95                            0.679806   
               asq_0.90                            0.712521   
               asq_0.85                            0.654930   
               asq_0.80                            0.614483   
               asq_default(0.7)                    0.517510   
triple         epigraphdb                               NaN   
               asq_0.99                                 NaN   
               asq_0.95                                 NaN   
               asq_0.90                                 NaN   
               asq_0.85                                 NaN   
               asq_0.80                                 NaN   
               asq_default(0.7)                         NaN   

evidence_type                    Insufficient (non-directional)  \
evidence_group filter_type                                        
assoc          epigraphdb                              0.912907   
               asq_0.99                                0.912907   
               asq_0.95                                0.843162   
               asq_0.90                                0.691900   
               asq_0.85                                0.671232   
               asq_0.80                                0.624270   
               asq_default(0.7)                        0.532437   
triple         epigraphdb                                   NaN   
               asq_0.99                                     NaN   
               asq_0.95                                     NaN   
               asq_0.90                                     NaN   
               asq_0.85                                     NaN   
               asq_0.80                                     NaN   
               asq_default(0.7)                             NaN   

evidence_type                    Addtional (directional)  
evidence_group filter_type                                
assoc          epigraphdb                            NaN  
               asq_0.99                              NaN  
               asq_0.95                              NaN  
               asq_0.90                              NaN  
               asq_0.85                              NaN  
               asq_0.80                              NaN  
               asq_default(0.7)                      NaN  
triple         epigraphdb                            NaN  
               asq_0.99                              NaN  
               asq_0.95                              NaN  
               asq_0.90                              NaN  
               asq_0.85                              NaN  
               asq_0.80                              NaN  
               asq_default(0.7)                      NaN

In [286]:
plot_data = (
    py_.chain(
        evidence_scores_df.groupby(["evidence_group", "evidence_type", "filter_type"])
        .apply(lambda df: df.to_dict(orient="records"))
        .to_list()
    )
    .filter(
        # keep group where group size > 10
        lambda coll: len(coll)
        >= 10
    )
    .map(pd.DataFrame)
    .map(lambda df: df if len(df) < 100 else df.sample(n=100))
    .apply(pd.concat)
    .value()
    .reset_index(drop=True)
)

# plot_data = plot_data[~plot_data["filter_type"].isin(["asq_0.99", "epigraphdb"])]
plot_data

,filter_type,evidence_type,evidence_group,evidence_score
0,asq_0.95,Supporting,assoc,2.159834
1,asq_0.95,Supporting,assoc,1.385091
2,asq_0.95,Supporting,assoc,1.520166
3,asq_0.95,Supporting,assoc,1.437106
4,asq_0.95,Supporting,assoc,2.077304
...,...,...,...,...
2992,asq_default(0.7),Reversal,triple,1.554742
2993,asq_default(0.7),Reversal,triple,1.104290
2994,asq_default(0.7),Reversal,triple,0.811546
2995,asq_default(0.7),Reversal,triple,1.416494


In [295]:
df = plot_data.loc[lambda df: df["evidence_type"].isin(["Supporting", "Reversal"])]
alt.Chart(df).mark_boxplot().encode(
    x=alt.X("evidence_score:Q"),
    y=alt.Y("filter_type:O", sort=filter_type_ordered),
    row=alt.Row("evidence_type:O", sort=["Supporting", "Reversal"]),
    column="evidence_group:O",
)

alt.Chart(...)